In [1]:
from utils.data_processing import calculate_monthly_stats, load_data
from utils.visualization import (plot_monthly_body_composition_stacked,
                                 plot_monthly_progress,
                                 plot_monthly_progress_boxplot,
                                 plot_variable_comparison,
                                 plot_daily_progress)
import polars as pl

In [2]:
df = load_data(
    './data/body_composition.csv',
    date_from='2025-04-01',
    date_to='2025-06-30',
    # device_id='aNyZC8jwH3'
)


In [ ]:
df.tail()

In [ ]:
fig_size = (12, 8) # (8, 7) for mobile, (12, 8) for desktop
plot_monthly_progress_boxplot(df, "weight", figsize=fig_size, y_label="Weight (kg)")
plot_monthly_progress_boxplot(df, "muscle_mass", figsize=fig_size, y_label="Muscle Mass (kg)")
plot_monthly_progress_boxplot(df, "body_fat_mass", figsize=fig_size, y_label="Body Fat Mass (kg)")
plot_monthly_progress_boxplot(df, "basal_metabolic_rate", figsize=fig_size, y_label="Basal Metabolic Rate (kcal)")

In [ ]:
fig_size = (8, 7) # (8, 7) for mobile, (12, 8) for desktop
plot_daily_progress(df, "muscle_mass", use_elapsed_days=False, figsize=fig_size)
plot_daily_progress(df, "body_fat_mass", use_elapsed_days=False, figsize=fig_size)
plot_daily_progress(df, "weight", use_elapsed_days=False, figsize=fig_size)

In [6]:
# df_monthly = calculate_monthly_stats(df)
# df_monthly.tail()

In [ ]:
plot_monthly_body_composition_stacked(
    df=df,
    title="Body Composition percentage",
    figsize=(8, 7)
)

In [ ]:
fig_size = (8, 7) # (8, 7) for mobile, (12, 8) for desktop
plot_variable_comparison(df, "weight", "muscle_mass", x_label="Weight (kg)", y_label="Muscle Mass (kg)", figsize=fig_size)
plot_variable_comparison(df, "weight", "body_fat_mass", x_label="Weight (kg)", y_label="Body Fat Mass (kg)", figsize=fig_size)
plot_variable_comparison(df, "weight", "basal_metabolic_rate", x_label="Weight (kg)", y_label="Basal Metabolic Rate (kcal)", figsize=fig_size)

In [9]:
# fig_size = (12, 8) # (8, 7) for mobile, (12, 8) for desktop
# plot_monthly_progress(df_monthly, "weight", y_label="Weight (kg)", figsize=fig_size )
# plot_monthly_progress(df_monthly, "muscle_mass", y_label="Muscle Mass (kg)", figsize=fig_size)
# plot_monthly_progress(df_monthly, "body_fat_mass", y_label="Body Fat Mass (kg)", positive_change=False, figsize=fig_size)
# plot_monthly_progress(df_monthly, "basal_metabolic_rate", y_label="Basal Metabolic Rate (kcal)", figsize=fig_size)